<h1 align="center"><font color="yellow">nn module - Pytorch: nn</font></h1>

<font color="yellow">Data Scientist.: PhD.Eddy Giusepe Chirinos Isidro</font>

```
nn module - Pytorch: nn
=======================

No PyTorch, o pacote nn serve para esse mesmo propósito (nível alto de Grafos computacionais brutos que são úteis para construir Redes Neurais). 
O pacote nn define um conjunto de Módulos, que são aproximadamente equivalentes a camadas de rede neural. Um módulo recebe tensores de entrada 
e calcula tensores de saída, mas também pode conter estado interno, como tensores contendo parâmetros que podem ser aprendidos. O pacote nn também 
define um conjunto de funções de perda úteis que são comumente usadas ao treinar redes neurais.

Link de estudo ---> https://pytorch.org/tutorials/beginner/pytorch_with_examples.html
```

In [1]:
import torch
import math

# Crie tensores para armazenar entradas e saídas:
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

Para este exemplo, a saída $y$ é uma função linear de $(x, x^2, x^3)$, então podemos considerá-la como uma rede neural de camada linear. Vamos preparar o tensor $(x, x^2, x^3)$.

In [2]:
p = torch.tensor([1, 2, 3])
p

tensor([1, 2, 3])

In [3]:
# x.unsqueeze(-1) --> Adiciona uma dimensão extra ao tensor 'x':
xx = x.unsqueeze(-1).pow(p)

xx

tensor([[ -3.1416,   9.8696, -31.0063],
        [ -3.1384,   9.8499, -30.9133],
        [ -3.1353,   9.8301, -30.8205],
        ...,
        [  3.1353,   9.8301,  30.8205],
        [  3.1384,   9.8499,  30.9133],
        [  3.1416,   9.8696,  31.0063]])

In [4]:
xx.shape

torch.Size([2000, 3])

No código acima, `x.unsqueeze(-1)` tem forma $(2000, 1)$ e $p$ tem forma $(3,)$, para este caso, a semântica de transmissão será aplicada para obter um tensor de forma $(2000, 3)$.



Use o pacote `nn` para definir nosso modelo como uma `sequência de camadas`. `nn.Sequential` é um Módulo que contém outros Módulos, e os aplica em seqüência para produzir sua saída. O Módulo Linear calcula a saída da entrada usando uma função linear e mantém os Tensores internos para seu peso e viés. A `camada Flatten` nivela a saída da camada linear para um `tensor 1D`, para corresponder à forma de `y`.


In [5]:
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)


O pacote `nn` também contém definições de funções de perda populares; neste caso, usaremos o `Mean Squared Error` (`MSE`) como nossa função de perda.

In [8]:
loss_fn = torch.nn.MSELoss(reduction='sum')

loss_fn

MSELoss()

In [9]:
learning_rate = 1e-6

In [10]:
for t in range(2000):

    """
    Forward pass: calcula 'y' previsto passando x para o modelo. 
    Os objetos do módulo substituem o operador __call__ para que você 
    possa chamá-los como funções. Ao fazer isso, você passa um tensor de 
    dados de entrada para o módulo e produz um tensor de dados de saída.
    """
    y_pred = model(xx)

    """
    Calculamos e printamos a loss. Passamos Tensores contendo os valores previstos e 
    verdadeiros de 'y', e a função de perda retorna um Tensor contendo a perda.
    """
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zere os gradientes antes de executar a passagem para trás (backward).
    model.zero_grad()

    """
    Backward pass: calcula o gradiente da Loss em relação a todos os parâmetros apreensíveis do modelo. 
    Internamente, os parâmetros de cada Módulo são armazenados em Tensores com 'require_grad=True', então 
    esta chamada calculará gradientes para todos os parâmetros que podem ser aprendidos no modelo.
    """
    loss.backward()

    """
    Atualize os pesos usando gradiente descendente. Cada parâmetro é um Tensor, então podemos acessar seus gradientes como fizemos antes.
    """
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad
            

99 1166.9737548828125
199 785.9876098632812
299 530.7454833984375
399 359.6208190917969
499 244.80453491210938
599 167.70742797851562
699 115.89563751220703
799 81.04689025878906
899 57.58669662475586
999 41.7790641784668
1099 31.11776351928711
1199 23.92055320739746
1299 19.05698585510254
1399 15.76708984375
1499 13.539349555969238
1599 12.02926254272461
1699 11.004531860351562
1799 10.308424949645996
1899 9.834992408752441
1999 9.51266098022461


In [13]:
# Você pode acessar a primeira camada do 'model' como acessar o primeiro item de uma lista:
linear_layer = model[0]
linear_layer


Linear(in_features=3, out_features=1, bias=True)

In [14]:
linear_layer_1 = model[1]
linear_layer_1

Flatten(start_dim=0, end_dim=1)

In [16]:
# Para a camada linear, seus parâmetros são armazenados como "weight" e "bias'.
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')


Result: y = 0.019588246941566467 + 0.838476300239563 x + -0.0033792981412261724 x^2 + -0.09073241800069809 x^3
